### Checking for CUDA availablilty

In [9]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
    print("Number of GPUs:", torch.cuda.device_count())
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Please check the installation.")


CUDA is available!
Number of GPUs: 1
GPU Name: NVIDIA GeForce GTX 1650


### Importing Useful Libraries

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


### Loading Pre-trained Resnet Model

In [5]:
# Load ResNet50 pre-trained model without the top layer (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of ResNet50
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for classification
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification (fake vs real)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


### Checking for Dataset Imbalaceing for agumentation

In [ ]:
#checking if data is balanced or unbalanced

# Count images in each class for train, validation, and test sets
import numpy as np

train_counts = np.unique(train_data.classes, return_counts=True)
val_counts = np.unique(val_data.classes, return_counts=True)
test_counts = np.unique(test_data.classes, return_counts=True)

# Print the counts
print(f"Training set: {dict(zip(train_data.class_indices.keys(), train_counts[1]))}")
print(f"Validation set: {dict(zip(val_data.class_indices.keys(), val_counts[1]))}")
print(f"Test set: {dict(zip(test_data.class_indices.keys(), test_counts[1]))}")


### Model Traing (Epoch = 10, Batch= 32, Img_size= 128)

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define constants
IMG_SIZE = 128  # Reduced image size for faster training
BATCH_SIZE = 32  # Increased batch size for fewer steps per epoch
EPOCHS = 10

# Data Generators with Augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255, validation_split=0.2,
    horizontal_flip=True, rotation_range=10)

train_data = datagen.flow_from_directory(
    "Dataset/Train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    "Dataset/Validation",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Model Definition using a lightweight MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze the base model

x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('best_deepfake_model.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stopping],
    verbose=1
)

# Evaluate the Model
test_loss, test_accuracy = model.evaluate(val_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Found 112002 images belonging to 2 classes.
Found 7885 images belonging to 2 classes.


c:\Users\alank\anaconda3\envs\deepfake_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3501/3501 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.7605 - loss: 0.5568

c:\Users\alank\anaconda3\envs\deepfake_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3501/3501 ━━━━━━━━━━━━━━━━━━━━ 936s 266ms/step - accuracy: 0.7605 - loss: 0.5568 - val_accuracy: 0.8205 - val_loss: 0.3937
Epoch 2/10
3501/3501 ━━━━━━━━━━━━━━━━━━━━ 1101s 314ms/step - accuracy: 0.8005 - loss: 0.4178 - val_accuracy: 0.8339 - val_loss: 0.3553
Epoch 3/10
3501/3501 ━━━━━━━━━━━━━━━━━━━━ 985s 281ms/step - accuracy: 0.8109 - loss: 0.4024 - val_accuracy: 0.8377 - val_loss: 0.3561
Epoch 4/10
3501/3501 ━━━━━━━━━━━━━━━━━━━━ 1025s 293ms/step - accuracy: 0.8151 - loss: 0.3933 - val_accuracy: 0.8375 - val_loss: 0.3494
Epoch 5/10
3501/3501 ━━━━━━━━━━━━━━━━━━━━ 1376s 393ms/step - accuracy: 0.8186 - loss: 0.3876 - val_accuracy: 0.8412 - val_loss: 0.3491
Epoch 6/10
3501/3501 ━━━━━━━━━━━━━━━━━━━━ 1379s 394ms/step - accuracy: 0.8229 - loss: 0.3787 - val_accuracy: 0.8417 - val_loss: 0.3457
Epoch 7/10
3501/3501 ━━━━━━━━━━━━━━━━━━━━ 1436s 410ms/step - accuracy: 0.8248 - loss: 0.3755 - val_accuracy: 0.8436 - val_loss: 0.3449
Epoch 8/10
3501/3501 ━━━━━━━━━━━━━━━━━━━━ 946s 270ms/step - accuracy

# Perfromance Metrices

In [ ]:
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Function to load and preprocess the image
def load_and_preprocess_image(img_path):
    img = cv2.imread(img_path)  # Read the image
    img = cv2.resize(img, (128, 128))  # Resize the image to match the model input size
    img = img.astype("float32") / 255.0  # Normalize the image
    img_array = img.reshape((1, 128, 128, 3))  # Add batch dimension
    return img, img_array

# Specify the model path
model_path = r'C:\Users\alank\OneDrive\Desktop\deepfake detection\best_deepfake_model.keras'

# Load the model
best_model = load_model(model_path)

# Provide the image path to test
img_path = 'C:\Users\alank\OneDrive\Desktop\deepfake detection\Dataset\Test\Real\real_1.jpg'  # Replace with your image path

# Load and preprocess the image
img, img_array = load_and_preprocess_image(img_path)

# Make a prediction
prediction = best_model.predict(img_array)[0][0]

# Classify the result based on the prediction value (threshold 0.5)
result = 'Real' if prediction >= 0.5 else 'Fake'
confidence = prediction if result == 'Real' else 1 - prediction  # Confidence score for either class

# Print the result and confidence
print(f"Prediction: {result}")
print(f"Confidence Score: {confidence:.4f}")

# Display the image with result and confidence score
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert color to RGB for display
plt.title(f"{result} - {confidence*100:.2f}% confidence")
plt.axis('off')  # Hide axes for better image display
plt.show()
